In [ ]:
import yaml
import matplotlib.pyplot as plt
import numpy as np

## All classes

In [ ]:
yaml_file = 'out/20230809_allclasses/optimization_data_cma.yaml'

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
data  

In [ ]:
def plot_map(mAP_values, cls=None):
    epochs = list(range(1, len(mAP_values)+1))
    plt.plot(epochs, mAP_values, linestyle='-', color='b')
    plt.xlabel('Epoch')
    plt.ylabel('mAP Value')
    if cls is not None:
        plt.title(f'mAP Evolution for {cls} category')
    else:
        plt.title('mAP Evolution')
    plt.grid(True)
    plt.show()

In [ ]:
classes = ['car', 'person', 'motorbike', 'motorcycle']
mAP_sum_per_index = np.zeros(len(data['car']['mAP']))
valid_classes = 0
for cls in classes:
    if cls in data.keys():
        mAP_values = data[cls]['mAP']
        plot_map(mAP_values, cls=cls)
        max_mAP = np.max(mAP_values)*100
        print(f'The maximum mAP achieved in {cls} category is {max_mAP:.2f} %')

        mAP_sum_per_index += mAP_values
        valid_classes += 1



In [ ]:
average_mAP_per_index = mAP_sum_per_index / valid_classes if valid_classes > 0 else np.zeros(len(data['car']['mAP']))
plot_map(average_mAP_per_index)
max_mAP = np.max(average_mAP_per_index)*100
print(f'The maximum mAP achieved is {max_mAP:.2f} %')
print(f'The final mAP achieved is {average_mAP_per_index[-1]*100:.2f} %')

The Hyperparameters combination with the best mAP:

In [ ]:
max_mAP_index = np.argmax(average_mAP_per_index)
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][max_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])

print(f'{text}mAP: {max_mAP:.2f}%')

The Hyperparameters combination with the final mAP:

In [ ]:
last_mAP_index = len(average_mAP_per_index)-1
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][last_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])

print(f'{text}mAP: {average_mAP_per_index[last_mAP_index]*100:.2f}%')

The code for inference

In [ ]:
from thirdparty.openISP.pipeline import Pipeline
from thirdparty.openISP.utils.yacs import Config
import skimage.io
import torch
import cv2
import matplotlib.pyplot as plt

model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, verbose=False)
class_colors = {
            "car": (128, 0, 128),  # Purple
            "person": (0, 0, 139),  # Dark red
            "motorbike": (255, 255, 0),  # Cyan
            "motorcycle": (255, 255, 0),  # Cyan
            "truck": (0, 139, 139)  # Yellow
        }

In [ ]:
def show_bounding_boxes(img, results, classes=["car"], min_conf=0.5, visualize_img=False, save_img=False, out_path=""):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    bboxes = results.xyxy[0]  # Obtener las bounding boxes
    for bbox in bboxes:
        class_name = results.names[int(bbox[5])]

        if (class_name in classes) and (bbox[4] >= min_conf):
            x1, y1, x2, y2 = map(int, bbox[:4])  # bounding box coordinates
            label = f"{class_name}: {bbox[4]:.2f}"  # Class and confidence score

            color = class_colors.get(class_name, (0, 0, 0)) # black if class not exists

            # Draw bounding box in the image
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 5)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 5)
    
    if visualize_img:
        plt.figure(figsize = (80,40))
        plt.axis("off")
        # convert color from CV2 BGR back to RGB
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.show()


    if save_img:
        # Calculate new dimensions (half of original size)
        new_width = img.shape[1] // 4
        new_height = img.shape[0] // 4
        # Resize the image
        resized_image = cv2.resize(img, (new_width, new_height))
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 70])



def inference(processed_img, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    results = model(processed_img)
    if visualize_img or save_img:
        show_bounding_boxes(processed_img, results, classes, min_conf, visualize_img, save_img, out_path)


def process_image(img_path, isp_conf, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    pipeline = Pipeline(isp_conf)
    bayer = skimage.io.imread(img_path).astype(np.uint8)
    data, _ = pipeline.execute(bayer, verbose=False)
    processed_img = data['output']
    #img_name = img_path.split('/')[-1]
    #img_out_path = out_path + img_name
    # Perform image detection and calculate IoU and mAP
    inference(processed_img, visualize_img=visualize_img, classes=classes, min_conf=min_conf, save_img=save_img, out_path=out_path)


def load_hyperparameters(dict_hyp, isp_config, dict_idx):
    for module in dict_hyp:
            for param in dict_hyp[module]:
                if type(dict_hyp[module][param][dict_idx]) == list:
                    with isp_config.unfreeze():
                        isp_config[module][param] = tuple(dict_hyp[module][param][dict_idx])
                else:
                    with isp_config.unfreeze():
                        isp_config[module][param] = dict_hyp[module][param][dict_idx]
    return isp_config

In [ ]:
config_file = "C:/Users/F36RB01/Documents/GitHub/isp_optimizer/configs/default_tuning.yaml"
img_path = "C:/Users/F36RB01/Documents/master/TFM/repos/raw_dataset.v4i.voc/valid/021.png"


isp_conf = Config(config_file)

In [ ]:
%matplotlib inline
process_image(img_path, isp_conf, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, max_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, last_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
for i in range(len(data['car']['mAP'])):
    new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, i)
    process_image(img_path, new_isp_config, visualize_img=False, classes=classes, save_img=True, out_path='processed_img/file%02d.jpg' % i)



In [ ]:
import os
import subprocess
os.chdir("processed_img")
subprocess.call([
    'ffmpeg', '-framerate', '8', '-i', 'file%02d.jpg', '-r', '30', '-pix_fmt', 'yuv420p',
    'video.mp4'
])

In [ ]:
from optimizer.optimizer import Optimizer



In [ ]:
cfg = Config('configs/main_config_notebook.yaml')
opt = Optimizer(cfg)

In [ ]:

load_hyperparameters(data["Hyperparameters"], opt.isp_conf, max_mAP_index)
print(opt.isp_conf)

In [ ]:
mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

In [ ]:
load_hyperparameters(data["Hyperparameters"], opt.isp_conf, last_mAP_index)
print(opt.isp_conf)

mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

## Cars

In [ ]:
yaml_file = 'out/20230811_car/out/optimization_data_cma.yaml'

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
data  

In [ ]:
def plot_map(mAP_values, cls=None):
    epochs = list(range(1, len(mAP_values)+1))
    plt.plot(epochs, mAP_values, linestyle='-', color='b')
    plt.xlabel('Epoch')
    plt.ylabel('mAP Value')
    if cls is not None:
        plt.title(f'mAP Evolution for {cls} category')
    else:
        plt.title('mAP Evolution')
    plt.grid(True)
    plt.show()

In [ ]:
classes = ['car', 'person', 'motorbike', 'motorcycle']
mAP_sum_per_index = np.zeros(len(data['car']['mAP']))
valid_classes = 0
for cls in classes:
    if cls in data.keys():
        mAP_values = data[cls]['mAP']
        plot_map(mAP_values, cls=cls)
        max_mAP = np.max(mAP_values)*100
        print(f'The maximum mAP achieved in {cls} category is {max_mAP:.2f} %')

        mAP_sum_per_index += mAP_values
        valid_classes += 1



In [ ]:
average_mAP_per_index = mAP_sum_per_index / valid_classes if valid_classes > 0 else np.zeros(len(data['car']['mAP']))
plot_map(average_mAP_per_index)
max_mAP = np.max(average_mAP_per_index)*100
print(f'The maximum mAP achieved is {max_mAP:.2f} %')
print(f'The final mAP achieved is {average_mAP_per_index[-1]*100:.2f} %')

The Hyperparameters combination with the best mAP:

In [ ]:
max_mAP_index = np.argmax(average_mAP_per_index)
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][max_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])

print(f'{text}mAP: {max_mAP:.2f}%')

The Hyperparameters combination with the final mAP:

In [ ]:
last_mAP_index = len(average_mAP_per_index)-1
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][last_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])

print(f'{text}mAP: {average_mAP_per_index[last_mAP_index]*100:.2f}%')

The code for inference

In [ ]:
from thirdparty.openISP.pipeline import Pipeline
from thirdparty.openISP.utils.yacs import Config
import skimage.io
import torch
import cv2
import matplotlib.pyplot as plt

model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, verbose=False)
class_colors = {
            "car": (128, 0, 128),  # Purple
            "person": (0, 0, 139),  # Dark red
            "motorbike": (255, 255, 0),  # Cyan
            "motorcycle": (255, 255, 0),  # Cyan
            "truck": (0, 139, 139)  # Yellow
        }

In [ ]:
def show_bounding_boxes(img, results, classes=["car"], min_conf=0.5, visualize_img=False, save_img=False, out_path=""):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    bboxes = results.xyxy[0]  # Obtener las bounding boxes
    for bbox in bboxes:
        class_name = results.names[int(bbox[5])]

        if (class_name in classes) and (bbox[4] >= min_conf):
            x1, y1, x2, y2 = map(int, bbox[:4])  # bounding box coordinates
            label = f"{class_name}: {bbox[4]:.2f}"  # Class and confidence score

            color = class_colors.get(class_name, (0, 0, 0)) # black if class not exists

            # Draw bounding box in the image
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 5)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 5)
    
    if visualize_img:
        plt.figure(figsize = (80,40))
        plt.axis("off")
        # convert color from CV2 BGR back to RGB
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.show()


    if save_img:
        # Calculate new dimensions (half of original size)
        new_width = img.shape[1] // 4
        new_height = img.shape[0] // 4
        # Resize the image
        resized_image = cv2.resize(img, (new_width, new_height))
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 70])



def inference(processed_img, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    results = model(processed_img)
    if visualize_img or save_img:
        show_bounding_boxes(processed_img, results, classes, min_conf, visualize_img, save_img, out_path)


def process_image(img_path, isp_conf, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    pipeline = Pipeline(isp_conf)
    bayer = skimage.io.imread(img_path).astype(np.uint8)
    data, _ = pipeline.execute(bayer, verbose=False)
    processed_img = data['output']
    #img_name = img_path.split('/')[-1]
    #img_out_path = out_path + img_name
    # Perform image detection and calculate IoU and mAP
    inference(processed_img, visualize_img=visualize_img, classes=classes, min_conf=min_conf, save_img=save_img, out_path=out_path)


def load_hyperparameters(dict_hyp, isp_config, dict_idx):
    for module in dict_hyp:
            for param in dict_hyp[module]:
                if type(dict_hyp[module][param][dict_idx]) == list:
                    with isp_config.unfreeze():
                        isp_config[module][param] = tuple(dict_hyp[module][param][dict_idx])
                else:
                    with isp_config.unfreeze():
                        isp_config[module][param] = dict_hyp[module][param][dict_idx]
    return isp_config

In [ ]:
config_file = "C:/Users/F36RB01/Documents/GitHub/isp_optimizer/configs/default_tuning.yaml"
img_path = "C:/Users/F36RB01/Documents/master/TFM/repos/raw_dataset.v4i.voc/valid/021.png"


isp_conf = Config(config_file)

In [ ]:
%matplotlib inline
process_image(img_path, isp_conf, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, max_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, last_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
for i in range(len(data['car']['mAP'])):
    new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, i)
    process_image(img_path, new_isp_config, visualize_img=False, classes=classes, save_img=True, out_path='processed_img/file%02d.jpg' % i)



In [ ]:
import os
import subprocess
os.chdir("processed_img")
subprocess.call([
    'ffmpeg', '-framerate', '8', '-i', 'file%02d.jpg', '-r', '30', '-pix_fmt', 'yuv420p',
    'video.mp4'
])

In [ ]:
from optimizer.optimizer import Optimizer



In [ ]:
cfg = Config('configs/main_config_notebook.yaml')
opt = Optimizer(cfg)

In [ ]:

load_hyperparameters(data["Hyperparameters"], opt.isp_conf, max_mAP_index)
print(opt.isp_conf)

In [ ]:
mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

In [ ]:
load_hyperparameters(data["Hyperparameters"], opt.isp_conf, last_mAP_index)
print(opt.isp_conf)

mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

## All classes Modifying mor hyperparameters

In [ ]:
yaml_file = 'out/20230821_allclasses/out/optimization_data_cma.yaml'

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
data  

In [ ]:
def plot_map(mAP_values, cls=None):
    epochs = list(range(1, len(mAP_values)+1))
    plt.plot(epochs, mAP_values, linestyle='-', color='b')
    plt.xlabel('Epoch')
    plt.ylabel('mAP Value')
    if cls is not None:
        plt.title(f'mAP Evolution for {cls} category')
    else:
        plt.title('mAP Evolution')
    plt.grid(True)
    plt.show()

In [ ]:
classes = ['car', 'person', 'motorbike', 'motorcycle']
mAP_sum_per_index = np.zeros(len(data['car']['mAP']))
valid_classes = 0
for cls in classes:
    if cls in data.keys():
        mAP_values = data[cls]['mAP']
        plot_map(mAP_values, cls=cls)
        max_mAP = np.max(mAP_values)*100
        print(f'The maximum mAP achieved in {cls} category is {max_mAP:.2f} %')

        mAP_sum_per_index += mAP_values
        valid_classes += 1



In [ ]:
average_mAP_per_index = mAP_sum_per_index / valid_classes if valid_classes > 0 else np.zeros(len(data['car']['mAP']))
plot_map(average_mAP_per_index)
max_mAP = np.max(average_mAP_per_index)*100
print(f'The maximum mAP achieved is {max_mAP:.2f} %')
print(f'The final mAP achieved is {average_mAP_per_index[-1]*100:.2f} %')

The Hyperparameters combination with the best mAP:

In [ ]:
max_mAP_index = np.argmax(average_mAP_per_index)
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][max_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])

print(f'{text}mAP: {max_mAP:.2f}%')

The Hyperparameters combination with the final mAP:

In [ ]:
last_mAP_index = len(average_mAP_per_index)-1
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][last_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])

print(f'{text}mAP: {average_mAP_per_index[last_mAP_index]*100:.2f}%')

The code for inference

In [ ]:
from thirdparty.openISP.pipeline import Pipeline
from thirdparty.openISP.utils.yacs import Config
import skimage.io
import torch
import cv2
import matplotlib.pyplot as plt

model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, verbose=False)
class_colors = {
            "car": (128, 0, 128),  # Purple
            "person": (0, 0, 139),  # Dark red
            "motorbike": (255, 255, 0),  # Cyan
            "motorcycle": (255, 255, 0),  # Cyan
            "truck": (0, 139, 139)  # Yellow
        }

In [ ]:
def show_bounding_boxes(img, results, classes=["car"], min_conf=0.5, visualize_img=False, save_img=False, out_path=""):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    bboxes = results.xyxy[0]  # Obtener las bounding boxes
    for bbox in bboxes:
        class_name = results.names[int(bbox[5])]

        if (class_name in classes) and (bbox[4] >= min_conf):
            x1, y1, x2, y2 = map(int, bbox[:4])  # bounding box coordinates
            label = f"{class_name}: {bbox[4]:.2f}"  # Class and confidence score

            color = class_colors.get(class_name, (0, 0, 0)) # black if class not exists

            # Draw bounding box in the image
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 5)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 5)
    
    if visualize_img:
        plt.figure(figsize = (80,40))
        plt.axis("off")
        # convert color from CV2 BGR back to RGB
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.show()


    if save_img:
        # Calculate new dimensions (half of original size)
        new_width = img.shape[1] // 4
        new_height = img.shape[0] // 4
        # Resize the image
        resized_image = cv2.resize(img, (new_width, new_height))
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 70])



def inference(processed_img, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    results = model(processed_img)
    if visualize_img or save_img:
        show_bounding_boxes(processed_img, results, classes, min_conf, visualize_img, save_img, out_path)


def process_image(img_path, isp_conf, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    pipeline = Pipeline(isp_conf)
    bayer = skimage.io.imread(img_path).astype(np.uint8)
    data, _ = pipeline.execute(bayer, verbose=False)
    processed_img = data['output']
    #img_name = img_path.split('/')[-1]
    #img_out_path = out_path + img_name
    # Perform image detection and calculate IoU and mAP
    inference(processed_img, visualize_img=visualize_img, classes=classes, min_conf=min_conf, save_img=save_img, out_path=out_path)


def load_hyperparameters(dict_hyp, isp_config, dict_idx):
    for module in dict_hyp:
            for param in dict_hyp[module]:
                if type(dict_hyp[module][param][dict_idx]) == list:
                    with isp_config.unfreeze():
                        isp_config[module][param] = tuple(dict_hyp[module][param][dict_idx])
                else:
                    with isp_config.unfreeze():
                        isp_config[module][param] = dict_hyp[module][param][dict_idx]
    return isp_config

In [ ]:
config_file = "C:/Users/F36RB01/Documents/GitHub/isp_optimizer/configs/default_tuning.yaml"
img_path = "C:/Users/F36RB01/Documents/master/TFM/repos/raw_dataset.v4i.voc/valid/021.png"


isp_conf = Config(config_file)

In [ ]:
%matplotlib inline
process_image(img_path, isp_conf, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, max_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, last_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
for i in range(len(data['car']['mAP'])):
    new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, i)
    process_image(img_path, new_isp_config, visualize_img=False, classes=classes, save_img=True, out_path='processed_img/file%02d.jpg' % i)



In [ ]:
import os
import subprocess
os.chdir("processed_img")
subprocess.call([
    'ffmpeg', '-framerate', '8', '-i', 'file%02d.jpg', '-r', '30', '-pix_fmt', 'yuv420p',
    'video.mp4'
])

In [ ]:
from optimizer.optimizer import Optimizer

cfg = Config('configs/main_config_notebook.yaml')
opt = Optimizer(cfg)

In [ ]:

load_hyperparameters(data["Hyperparameters"], opt.isp_conf, max_mAP_index)
print(opt.isp_conf)

In [ ]:
mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

In [ ]:
load_hyperparameters(data["Hyperparameters"], opt.isp_conf, last_mAP_index)
print(opt.isp_conf)

mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

## All classes Last results of 04/09/2023 using training set of 71 images

In [ ]:
yaml_file = 'out/20230904_allclasses/out/optimization_data_cma.yaml'

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
data  

In [ ]:
def plot_map(mAP_values, cls=None):
    epochs = list(range(1, len(mAP_values)+1))
    plt.plot(epochs, mAP_values, linestyle='-', color='b')
    plt.xlabel('Epoch')
    plt.ylabel('mAP Value')
    if cls is not None:
        plt.title(f'mAP Evolution for {cls} category')
    else:
        plt.title('mAP Evolution')
    plt.grid(True)
    plt.show()

In [ ]:
classes = ['car', 'person', 'motorbike', 'motorcycle']
mAP_sum_per_index = np.zeros(len(data['car']['mAP']))
valid_classes = 0
for cls in classes:
    if cls in data.keys():
        mAP_values = data[cls]['mAP']
        plot_map(mAP_values, cls=cls)
        max_mAP = np.max(mAP_values)*100
        print(f'The maximum mAP achieved in {cls} category is {max_mAP:.2f} %')

        mAP_sum_per_index += mAP_values
        valid_classes += 1



In [ ]:
average_mAP_per_index = mAP_sum_per_index / valid_classes if valid_classes > 0 else np.zeros(len(data['car']['mAP']))
plot_map(average_mAP_per_index)
max_mAP = np.max(average_mAP_per_index)*100
print(f'The maximum mAP achieved is {max_mAP:.2f} %')
print(f'The final mAP achieved is {average_mAP_per_index[-1]*100:.2f} %')

In [ ]:
max_idx = 283*11
reduced_average_mAP_per_index = average_mAP_per_index[0:max_idx]
reduced_average_mAP_2 = np.zeros(283)
for i in range(0,len(reduced_average_mAP_per_index)-10, 11):
    #temp_list = reduced_average_mAP_per_index[i:i+11]
    #print(temp_list)
    reduced_average_mAP_2[int(i/11)] = np.max(reduced_average_mAP_per_index[i:i+11])

#print(reduced_average_mAP_2)
plot_map(reduced_average_mAP_2)
max_mAP = np.max(reduced_average_mAP_2)*100
print(f'The maximum mAP achieved is {max_mAP:.2f} %')
print(f'The final mAP achieved is {reduced_average_mAP_2[-1]*100:.2f} %')

The Hyperparameters combination with the best mAP:

In [ ]:
max_mAP_index = np.argmax(average_mAP_per_index)
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][max_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][max_mAP_index])

print(f'{text}mAP: {max_mAP:.2f}%')

The Hyperparameters combination with the final mAP:

In [ ]:
last_mAP_index = len(average_mAP_per_index)-1
text = ''
for module in data['Hyperparameters']:
    for param in data['Hyperparameters'][module]:
        if type(data['Hyperparameters'][module][param][last_mAP_index]) == list:
            text += '{}:{}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])
        else:
            text += '{}:{:.2f}\t'.format(param, data['Hyperparameters'][module][param][last_mAP_index])

print(f'{text}mAP: {average_mAP_per_index[last_mAP_index]*100:.2f}%')

The code for inference

In [ ]:
from thirdparty.openISP.pipeline import Pipeline
from thirdparty.openISP.utils.yacs import Config
import skimage.io
import torch
import cv2
import matplotlib.pyplot as plt

model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True, verbose=False)
class_colors = {
            "car": (128, 0, 128),  # Purple
            "person": (0, 0, 139),  # Dark red
            "motorbike": (255, 255, 0),  # Cyan
            "motorcycle": (255, 255, 0),  # Cyan
            "truck": (0, 139, 139)  # Yellow
        }

In [ ]:
def show_bounding_boxes(img, results, classes=["car"], min_conf=0.5, visualize_img=False, save_img=False, out_path=""):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    bboxes = results.xyxy[0]  # Obtener las bounding boxes
    for bbox in bboxes:
        class_name = results.names[int(bbox[5])]

        if (class_name in classes) and (bbox[4] >= min_conf):
            x1, y1, x2, y2 = map(int, bbox[:4])  # bounding box coordinates
            label = f"{class_name}: {bbox[4]:.2f}"  # Class and confidence score

            color = class_colors.get(class_name, (0, 0, 0)) # black if class not exists

            # Draw bounding box in the image
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 5)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 5)
    
    if visualize_img:
        plt.figure(figsize = (80,40))
        plt.axis("off")
        # convert color from CV2 BGR back to RGB
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.show()


    if save_img:
        # Calculate new dimensions (half of original size)
        new_width = img.shape[1] // 4
        new_height = img.shape[0] // 4
        # Resize the image
        resized_image = cv2.resize(img, (new_width, new_height))
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 70])



def inference(processed_img, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    results = model(processed_img)
    if visualize_img or save_img:
        show_bounding_boxes(processed_img, results, classes, min_conf, visualize_img, save_img, out_path)


def process_image(img_path, isp_conf, visualize_img=False, classes=['car', 'motorbike', 'person'], min_conf=0.5, save_img=False, out_path=''):
    pipeline = Pipeline(isp_conf)
    bayer = skimage.io.imread(img_path).astype(np.uint8)
    data, _ = pipeline.execute(bayer, verbose=False)
    processed_img = data['output']
    #img_name = img_path.split('/')[-1]
    #img_out_path = out_path + img_name
    # Perform image detection and calculate IoU and mAP
    inference(processed_img, visualize_img=visualize_img, classes=classes, min_conf=min_conf, save_img=save_img, out_path=out_path)


def load_hyperparameters(dict_hyp, isp_config, dict_idx):
    for module in dict_hyp:
            for param in dict_hyp[module]:
                if type(dict_hyp[module][param][dict_idx]) == list:
                    with isp_config.unfreeze():
                        isp_config[module][param] = tuple(dict_hyp[module][param][dict_idx])
                else:
                    with isp_config.unfreeze():
                        isp_config[module][param] = dict_hyp[module][param][dict_idx]
    return isp_config

In [ ]:
config_file = "C:/Users/F36RB01/Documents/GitHub/isp_optimizer/configs/default_tuning.yaml"
img_path = "C:/Users/F36RB01/Documents/master/TFM/repos/raw_dataset.v4i.voc/valid/021.png"


isp_conf = Config(config_file)

In [ ]:
%matplotlib inline
process_image(img_path, isp_conf, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, max_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, last_mAP_index)
process_image(img_path, new_isp_config, visualize_img=True, classes=classes)

In [ ]:
for i in range(len(data['car']['mAP'])):
    new_isp_config = load_hyperparameters(data["Hyperparameters"], isp_conf, i)
    process_image(img_path, new_isp_config, visualize_img=False, classes=classes, save_img=True, out_path='processed_img/file%02d.jpg' % i)



In [ ]:
import os
import subprocess
os.chdir("processed_img")
subprocess.call([
    'ffmpeg', '-framerate', '8', '-i', 'file%02d.jpg', '-r', '30', '-pix_fmt', 'yuv420p',
    'video.mp4'
])

In [ ]:
from optimizer.optimizer import Optimizer

cfg = Config('configs/main_config_notebook.yaml')
opt = Optimizer(cfg)

In [ ]:

load_hyperparameters(data["Hyperparameters"], opt.isp_conf, max_mAP_index)
print(opt.isp_conf)

In [ ]:
mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')

In [ ]:
load_hyperparameters(data["Hyperparameters"], opt.isp_conf, last_mAP_index)
print(opt.isp_conf)

mean_iou, mean_mAP, end_time = opt.batch_image_processing()
print(f'IoU: {mean_iou:.3f} \t mAP: {mean_mAP:.3f} \t Computation Time (s): {end_time:.5f}')